In [1]:
import pandas as pd
from pandas import json_normalize
from datetime import date
pd.set_option('display.max_columns', None)

In [2]:
df_author_raw = catalog.load('raw/openalex/author#parquet')

[09/28/25 15:30:14] INFO     Loading data from raw/openalex/author#parquet (ParquetDataset)...  ]8;id=795037;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=736815;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

# Profiling

In [3]:
df_author_raw.dtypes


id                           object
orcid                        object
display_name                 object
display_name_alternatives    object
works_count                   int64
cited_by_count                int64
summary_stats                object
ids                          object
affiliations                 object
last_known_institutions      object
topics                       object
topic_share                  object
x_concepts                   object
counts_by_year               object
works_api_url                object
updated_date                 object
created_date                 object
load_datetime                object
dtype: object

In [4]:
df_author_raw

,id,orcid,display_name,display_name_alternatives,works_count,cited_by_count,summary_stats,ids,affiliations,last_known_institutions,topics,topic_share,x_concepts,counts_by_year,works_api_url,updated_date,created_date,load_datetime
0,https://openalex.org/A5013845636,https://orcid.org/0000-0002-6764-4789,P. H. Hansen,"[J B. Hansen, M. C. Hansen, J R H Hansen, Fran...",2887,133693,"{'2yr_mean_citedness': 2.525911708253359, 'h_i...",{'openalex': 'https://openalex.org/A5013845636...,"[{'institution': {'country_code': 'DK', 'displ...","[{'country_code': 'DK', 'display_name': 'Unive...","[{'count': 2531, 'display_name': 'Particle phy...",[{'display_name': 'High-Energy Particle Collis...,"[{'display_name': 'Physics', 'id': 'https://op...","[{'cited_by_count': 6613, 'oa_works_count': 49...",https://api.openalex.org/works?filter=author.i...,2025-09-07T11:01:14.374671,2023-07-21,2025-09-08
1,https://openalex.org/A5049875695,https://orcid.org/0000-0002-1653-1303,M. Erdmann,"[M. Erdmann, W. Erdmann, J. Erdmann, Martin Er...",2675,136872,"{'2yr_mean_citedness': 3.5545722713864305, 'h_...",{'openalex': 'https://openalex.org/A5049875695...,"[{'institution': {'country_code': 'DE', 'displ...","[{'country_code': 'DE', 'display_name': 'RWTH ...","[{'count': 2183, 'display_name': 'Particle phy...",[{'display_name': 'High-Energy Particle Collis...,"[{'display_name': 'Physics', 'id': 'https://op...","[{'cited_by_count': 4350, 'oa_works_count': 47...",https://api.openalex.org/works?filter=author.i...,2025-09-02T11:40:42.494453,2023-07-21,2025-09-08
2,https://openalex.org/A5046725106,https://orcid.org/0000-0001-7951-0166,C. Padilla Aranda,"[Carolina Rodríguez Padilla, C. Padilla, C. Pa...",2280,125614,"{'2yr_mean_citedness': 4.683139534883721, 'h_i...",{'openalex': 'https://openalex.org/A5046725106...,"[{'institution': {'country_code': 'ES', 'displ...","[{'country_code': 'ES', 'display_name': 'Barce...","[{'count': 1946, 'display_name': 'Particle phy...",[{'display_name': 'High-Energy Particle Collis...,"[{'display_name': 'Physics', 'id': 'https://op...","[{'cited_by_count': 4435, 'oa_works_count': 50...",https://api.openalex.org/works?filter=author.i...,2025-09-01T14:05:29.806983,2023-07-21,2025-09-08
3,https://openalex.org/A5087325828,https://orcid.org/0000-0002-3021-0258,F. Barreiro,"[F. Alonso, F H Barreiro, J. Costa, J. Barreir...",2178,122889,"{'2yr_mean_citedness': 4.641025641025641, 'h_i...",{'openalex': 'https://openalex.org/A5087325828...,"[{'institution': {'country_code': 'ES', 'displ...","[{'country_code': 'ES', 'display_name': 'Unive...","[{'count': 2088, 'display_name': 'Particle phy...",[{'display_name': 'High-Energy Particle Collis...,"[{'display_name': 'Physics', 'id': 'https://op...","[{'cited_by_count': 5790, 'oa_works_count': 45...",https://api.openalex.org/works?filter=author.i...,2025-08-29T22:33:14.140360,2023-07-21,2025-09-08
4,https://openalex.org/A5062435541,https://orcid.org/0000-0001-6322-6195,A. T. Doyle,"[Tony Doyle, Anthony T. Doyle, A. Doyle, T. Do...",2175,117700,"{'2yr_mean_citedness': 3.6802325581395348, 'h_...",{'openalex': 'https://openalex.org/A5062435541...,"[{'institution': {'country_code': 'GB', 'displ...","[{'country_code': 'GB', 'display_name': 'Unive...","[{'count': 1937, 'display_name': 'Particle phy...",[{'display_name': 'High-Energy Particle Collis...,"[{'display_name': 'Physics', 'id': 'https://op...","[{'cited_by_count': 2955, 'oa_works_count': 48...",https://api.openalex.org/works?filter=author.i...,2025-09-08T09:37:28.667590,2023-07-21,2025-09-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,https://openalex.org/A5027210888,None,Virpi Sinikka Niemelä,"[V. S. Niemelä, Virpi Sinikka Niemelä, V. S. N...",395,2044,"{'2yr_mean_citedness': 0.0, 'h_index': 26, 'i1...",{'openalex': 'https://openalex.org/A5027210888...,"[{'institution': {'country_code': 'AR', 'displ...","[{'country_code': 'AR', 'display_name': 'Insti...","[{'count': 200, 'display_name': 'Astronomical ...",[{'dis

# Nodo

In [5]:
def openalex_land_author(df: pd.DataFrame)-> pd.DataFrame:
    
    df_author = df.drop(
        columns=[
            'display_name_alternatives', 
            'summary_stats', 
            'ids', 
            'affiliations', 
            'last_known_institutions', 
            'topics', 
            'topic_share', 
            'x_concepts', 
            'counts_by_year', 
            'works_api_url'
        ]
    )

    df_author = df_author.convert_dtypes()
    df_author['load_datetime'] = date.today()
    
    return df_author



# Ejecuto nodo

In [6]:
df_author = openalex_land_author(df_author_raw)

# Resultados

In [7]:
df_author

,id,orcid,display_name,works_count,cited_by_count,updated_date,created_date,load_datetime
0,https://openalex.org/A5013845636,https://orcid.org/0000-0002-6764-4789,P. H. Hansen,2887,133693,2025-09-07T11:01:14.374671,2023-07-21,2025-09-28
1,https://openalex.org/A5049875695,https://orcid.org/0000-0002-1653-1303,M. Erdmann,2675,136872,2025-09-02T11:40:42.494453,2023-07-21,2025-09-28
2,https://openalex.org/A5046725106,https://orcid.org/0000-0001-7951-0166,C. Padilla Aranda,2280,125614,2025-09-01T14:05:29.806983,2023-07-21,2025-09-28
3,https://openalex.org/A5087325828,https://orcid.org/0000-0002-3021-0258,F. Barreiro,2178,122889,2025-08-29T22:33:14.140360,2023-07-21,2025-09-28
4,https://openalex.org/A5062435541,https://orcid.org/0000-0001-6322-6195,A. T. Doyle,2175,117700,2025-09-08T09:37:28.667590,2023-07-21,2025-09-28
...,...,...,...,...,...,...,...,...
20,https://openalex.org/A5027210888,<NA>,Virpi Sinikka Niemelä,395,2044,2025-09-07T04:09:19.897516,2023-07-21,2025-09-28
21,https://openalex.org/A5070009291,https://orcid.org/0000-0003-3968-6604,Nancy M. Greco,394,872,2025-08-28T01:47:35.314049,2023-07-21,2025-09-28
22,https://openalex.org/A5058717609,https://orcid.org/0000-0003-2771-7805,L. G. Althaus,393,8090,2025-08-31T01:13:54.100158,2023-07-21,2025-09-28
23,https://openalex.org/A5019082243,https://orcid.org/0000-0003-2422-6322,Marisa Raquel De Giusti,390,719,2025-09-04T07:12:56.702974,2023-07-21,2025-09-28
